"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
### Alias : unit_table.py & Last Modded : 2023.07.09. ###
Coded with Python 3.10 Grammar by Jin, Hojin
Description : FOS Controller Prototype
"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""

#### Basic Settings

In [1]:
import asyncio
import socket
import json

import pandas as pd

HOST = "127.0.0.1"
PORT = 8080
SIZE = 4096

In [2]:
def recv():
    global client_socket
    fragments = []
    while True:
        chunk = client_socket.recv(SIZE)
        if not chunk:
            raise Exception("Socket connection closed unexpectedly.")
        fragments.append(chunk)
        if len(chunk) < SIZE:
            break
    return b''.join(fragments).decode()

#### Server Open

In [3]:
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind((HOST, PORT))
server_socket.listen()

#### Client Accept

In [4]:
client_socket, client_addr = server_socket.accept()

In [5]:
msg = recv()
msg

'!CONNECTED!'

#### Game Start

#### Round 1

In [6]:
client_socket.sendall("/start".encode())

In [7]:
msg = pd.DataFrame.from_dict(data=json.loads(recv()), orient='index')
msg

,0
code,200
message,Round 1 is ready.
data,{'round': 1}


In [8]:
client_socket.sendall("/data/aircraft_specsheet".encode())

In [9]:
spec = pd.DataFrame.from_dict(data=json.loads(recv()), orient='index')
spec

,0
code,200
message,Success
data,"{""D1"": {""Aircraft Type"": ""Drone"", ""Velocity"": ..."


In [10]:
client_socket.sendall("/data/target_list".encode())

In [11]:
targets = pd.DataFrame.from_dict(data=json.loads(recv()), orient='index')
targets

,0
code,200
message,Success
data,"{""Targets"": {""T1"": {""latitude"": 280, ""longitud..."
time,0600


In [12]:
Target_list = pd.DataFrame.from_dict(data=json.loads(targets[0]['data']), orient='columns')

In [13]:
client_socket.sendall("/data/unit_table".encode())

In [14]:
units = pd.DataFrame.from_dict(data=json.loads(recv()), orient='index')
units

,0
code,200
message,Success
data,"{""D1-A"": {""Ordered"": false, ""Available"": true,..."
time,0600


In [15]:
Target_dict = json.loads(targets[0]['data'])
Target_dict
Target_list_temp=[]
random_target = 'T0'
for i in range(1,10):
    random_target_temp = 'T'+f'{i}'
    if(Target_dict['Targets'][random_target_temp]['probability'] != 0):
        Target_list_temp.append(Target_dict['Targets'][random_target_temp])
        random_target = random_target_temp
Lake_info = Target_dict['Lakes']['L1']

In [16]:
#좌표 한칸당 1km로 가정
import math
import collections
lake_lat = Lake_info['latitude']
lake_long = Lake_info['longitude']

base_list = ['A', 'B', 'C']
base_lat ={}
base_long = {}
unit_dict = json.loads(units[0]['data'])
aircraft_info = json.loads(spec[0]['data'])
time_dict = {}
weighted_cost = {}
for i in base_list:
    base_lat[i] = Target_dict['Bases'][i]['latitude']
    base_long[i] = Target_dict['Bases'][i]['longitude']

In [17]:
def select_aircraft(Target_list, round_num):
    result_aircraft = []
    for i in range(0, round_num):
        target_lat = Target_list[i]['latitude']
        target_long = Target_list[i]['longitude']
    
        for name in unit_dict:
            result_time = 0
            aircraft_name = name[0:2]
            velocity = aircraft_info[aircraft_name]['Velocity']
            base_name = unit_dict[name]['Base']
            water_need = 100 - unit_dict[name]['Current Water']
            water_fulltime = aircraft_info[aircraft_name]['Water Tank']
            Time_water = water_fulltime*(water_need/100)
            if(Time_water != 0):
                result_time = aircraft_info[aircraft_name]['ETRDY'] +Time_water + (math.sqrt((lake_lat - base_lat[base_name])**2 + (lake_long - base_long[base_name])**2)/velocity)*60 + (math.sqrt((target_lat - lake_lat)**2 + (target_long - lake_long)**2)/velocity)*60
            else:
                result_time = aircraft_info[aircraft_name]['ETRDY'] + (math.sqrt((target_lat - base_lat[base_name])**2 + (target_long - base_long[base_name])**2)/velocity)*60
            time_dict[name] = result_time
            weight = (100 / aircraft_info[aircraft_name]['Possibility'])**2
            
            weighted_cost[name] = int(((aircraft_info[aircraft_name]['Cost']*weight/90.0)**1.16 + 10*math.log10((result_time*weight+2)**2)) * 100)
        
        sorted_cost = sorted(weighted_cost.items(), key=lambda x: x[1], reverse=False)
            
        result_aircraft.append(sorted_cost)
    return result_aircraft

In [18]:
xml = f'''<operations>
    <order>
        <time>0601</time>
        <base>{unit_dict[select_aircraft(Target_list_temp,1)[0][0][0]]['Base']}</base>
        <aircraft_type>{aircraft_info[select_aircraft(Target_list_temp,1)[0][0][0][0:2]]['Aircraft Type']}</aircraft_type>
        <track_number>{select_aircraft(Target_list_temp,1)[0][0][0]}</track_number>
        <mission_type>1</mission_type>
        <course>{random_target}</course>
    </order>
    <order>
        <time>0602</time>
        <base>{unit_dict[select_aircraft(Target_list_temp,1)[0][1][0]]['Base']}</base>
        <aircraft_type>{aircraft_info[select_aircraft(Target_list_temp,1)[0][1][0][0:2]]['Aircraft Type']}</aircraft_type>
        <track_number>{select_aircraft(Target_list_temp,1)[0][1][0]}</track_number>
        <mission_type>1</mission_type>
        <course>{random_target}</course>
    </order>
</operations>'''

In [19]:
client_socket.sendall(f"/order/{xml}".encode())

order_result = pd.DataFrame.from_dict(data=json.loads(recv()), orient='index')
order_result

,0
code,200
message,Success


In [20]:
client_socket.sendall("/data/unit_table".encode())

In [21]:
units = pd.DataFrame.from_dict(data=json.loads(recv()), orient='index')
units

,0
code,200
message,Success
data,"{""D1-A"": {""Ordered"": false, ""Available"": true,..."
time,0601


In [22]:
xml_skip = '''<operations>
            </operations>'''

In [23]:
aircraft_number = 2
while(True):
    check = 0
    trigger = 0
    client_socket.sendall(f"/order/{xml_skip}".encode())

    order_result = pd.DataFrame.from_dict(data=json.loads(recv()), orient='index')
    order_result
    
    client_socket.sendall("/data/target_list".encode())

    targets = pd.DataFrame.from_dict(data=json.loads(recv()), orient='index')
    targets
    
    try:
        Target_dict = json.loads(targets[0]['data'])
        Target_dict
    except:
        break
    
    Target_list_temp=[]
    random_target = []
    
    for i in range(1,10):
        random_target_temp = 'T'+f'{i}'
        if(Target_dict['Targets'][random_target_temp]['threat'] == 0):
            check = check + 1

        if(Target_dict['Targets'][random_target_temp]['probability'] != 0 and Target_dict['Targets'][random_target_temp]['targeted'] != True):
            aircraft_number = aircraft_number + 1
            if (aircraft_number >= 30):
                aircraft_number = aircraft_number % 30
            trigger = 1
            Target_list_temp.append(Target_dict['Targets'][random_target_temp])
            random_target.append(random_target_temp)
    if(check == 9):
        break
    if trigger == 1:    
        client_socket.sendall("/data/unit_table".encode())

        units = pd.DataFrame.from_dict(data=json.loads(recv()), orient='index')
        units
        
        xml = f'''<operations>
        <order>
            <time>{"0"+str(int(units[0]['time'])+1)}</time>
            <base>{unit_dict[select_aircraft(Target_list_temp,1)[0][aircraft_number][0]]['Base']}</base>
            <aircraft_type>{aircraft_info[select_aircraft(Target_list_temp,1)[0][aircraft_number][0][0:2]]['Aircraft Type']}</aircraft_type>
            <track_number>{select_aircraft(Target_list_temp,1)[0][aircraft_number][0]}</track_number>
            <mission_type>1</mission_type>
            <course>{random_target[0]}</course>
        </order>
        </operations>'''

        client_socket.sendall(f"/order/{xml}".encode())

        order_result = pd.DataFrame.from_dict(data=json.loads(recv()), orient='index')
        order_result

        client_socket.sendall("/data/unit_table".encode())

        units = pd.DataFrame.from_dict(data=json.loads(recv()), orient='index')
        units

In [24]:
client_socket.sendall("/result".encode())

msg = pd.DataFrame.from_dict(data=json.loads(recv()), orient='index')
msg

,0
code,200
message,Round 1 is finished.
data,"{'round': 1, 'is_win': True, 'score': 3669}"


#### Round 2

In [25]:
client_socket.sendall("/start".encode())

msg = pd.DataFrame.from_dict(data=json.loads(recv()), orient='index')
msg

,0
code,200
message,Round 2 is ready.
data,{'round': 2}


In [26]:
client_socket.sendall("/data/unit_table".encode())

units = pd.DataFrame.from_dict(data=json.loads(recv()), orient='index')
units

,0
code,200
message,Success
data,"{""D1-A"": {""Ordered"": false, ""Available"": true,..."
time,0600


In [27]:
client_socket.sendall("/data/target_list".encode())

targets = pd.DataFrame.from_dict(data=json.loads(recv()), orient='index')
targets

,0
code,200
message,Success
data,"{""Targets"": {""T1"": {""latitude"": 280, ""longitud..."
time,0600


In [28]:
Target_list = pd.DataFrame.from_dict(data=json.loads(targets[0]['data']), orient='columns')

In [29]:
client_socket.sendall("/data/unit_table".encode())

units = pd.DataFrame.from_dict(data=json.loads(recv()), orient='index')
units

,0
code,200
message,Success
data,"{""D1-A"": {""Ordered"": false, ""Available"": true,..."
time,0600


In [30]:
Target_dict = json.loads(targets[0]['data'])
Target_dict
Target_list_temp=[]
random_target = []
for i in range(1,10):
    random_target_temp = 'T'+f'{i}'
    if(Target_dict['Targets'][random_target_temp]['probability'] != 0):
        Target_list_temp.append(Target_dict['Targets'][random_target_temp])
        random_target.append(random_target_temp)
Lake_info = Target_dict['Lakes']['L1']

In [31]:
lake_lat = Lake_info['latitude']
lake_long = Lake_info['longitude']

base_list = ['A', 'B', 'C']
base_lat ={}
base_long = {}
unit_dict = json.loads(units[0]['data'])
aircraft_info = json.loads(spec[0]['data'])
time_dict = {}
weighted_cost = {}
for i in base_list:
    base_lat[i] = Target_dict['Bases'][i]['latitude']
    base_long[i] = Target_dict['Bases'][i]['longitude']

In [32]:
client_socket.sendall("/data/unit_table".encode())

units = pd.DataFrame.from_dict(data=json.loads(recv()), orient='index')
units

,0
code,200
message,Success
data,"{""D1-A"": {""Ordered"": false, ""Available"": true,..."
time,0600


In [33]:
xml = f'''<operations>
    <order>
        <time>0601</time>
        <base>{unit_dict[select_aircraft(Target_list_temp,2)[0][0][0]]['Base']}</base>
        <aircraft_type>{aircraft_info[select_aircraft(Target_list_temp,2)[0][0][0][0:2]]['Aircraft Type']}</aircraft_type>
        <track_number>{select_aircraft(Target_list_temp,2)[0][0][0]}</track_number>
        <mission_type>1</mission_type>
        <course>{random_target[0]}</course>
    </order>
    <order>
        <time>0602</time>
        <base>{unit_dict[select_aircraft(Target_list_temp,2)[0][1][0]]['Base']}</base>
        <aircraft_type>{aircraft_info[select_aircraft(Target_list_temp,2)[0][1][0][0:2]]['Aircraft Type']}</aircraft_type>
        <track_number>{select_aircraft(Target_list_temp,2)[0][1][0]}</track_number>
        <mission_type>1</mission_type>
        <course>{random_target[1]}</course>
    </order>
    <order>
        <time>0603</time>
        <base>{unit_dict[select_aircraft(Target_list_temp,2)[0][2][0]]['Base']}</base>
        <aircraft_type>{aircraft_info[select_aircraft(Target_list_temp,2)[0][2][0][0:2]]['Aircraft Type']}</aircraft_type>
        <track_number>{select_aircraft(Target_list_temp,2)[0][2][0]}</track_number>
        <mission_type>1</mission_type>
        <course>{random_target[0]}</course>
    </order>
    <order>
        <time>0604</time>
        <base>{unit_dict[select_aircraft(Target_list_temp,2)[0][3][0]]['Base']}</base>
        <aircraft_type>{aircraft_info[select_aircraft(Target_list_temp,2)[0][3][0][0:2]]['Aircraft Type']}</aircraft_type>
        <track_number>{select_aircraft(Target_list_temp,2)[0][3][0]}</track_number>
        <mission_type>1</mission_type>
        <course>{random_target[1]}</course>
    </order>
</operations>'''

In [34]:
client_socket.sendall(f"/order/{xml}".encode())

order_result = pd.DataFrame.from_dict(data=json.loads(recv()), orient='index')
order_result

,0
code,200
message,Success


In [35]:
xml_skip = '''<operations>
            </operations>'''

In [36]:
client_socket.sendall("/data/unit_table".encode())

units = pd.DataFrame.from_dict(data=json.loads(recv()), orient='index')
units

,0
code,200
message,Success
data,"{""D1-A"": {""Ordered"": false, ""Available"": true,..."
time,0601


In [37]:
aircraft_number = 4
iter_num_list = []
while True:
    check = 0
    trigger = 0
    client_socket.sendall(f"/order/{xml_skip}".encode())

    order_result = pd.DataFrame.from_dict(data=json.loads(recv()), orient='index')
    order_result

    client_socket.sendall("/data/target_list".encode())

    targets = pd.DataFrame.from_dict(data=json.loads(recv()), orient='index')
    targets

    try:
        Target_dict = json.loads(targets[0]['data'])
        Target_dict
    except:
        break
    
    Target_list_temp=[]
    random_target = []
    for i in range(1,10):
        random_target_temp = 'T'+f'{i}'
        if(Target_dict['Targets'][random_target_temp]['threat'] == 0):
            check = check + 1

        if(Target_dict['Targets'][random_target_temp]['probability'] != 0 and Target_dict['Targets'][random_target_temp]['targeted'] != True):
            aircraft_number = aircraft_number + 1
            if (aircraft_number >= 30):
                aircraft_number = aircraft_number % 30
            trigger = 1
            Target_list_temp.append(Target_dict['Targets'][random_target_temp])
            random_target.append(random_target_temp)
    if(check == 9):
        break
    if trigger == 1:    
        client_socket.sendall("/data/unit_table".encode())

        units = pd.DataFrame.from_dict(data=json.loads(recv()), orient='index')
        units
        
        xml = f'''<operations>
        <order>
            <time>{"0"+str(int(units[0]['time'])+1)}</time>
            <base>{unit_dict[select_aircraft(Target_list_temp,1)[0][aircraft_number][0]]['Base']}</base>
            <aircraft_type>{aircraft_info[select_aircraft(Target_list_temp,1)[0][aircraft_number][0][0:2]]['Aircraft Type']}</aircraft_type>
            <track_number>{select_aircraft(Target_list_temp,1)[0][aircraft_number][0]}</track_number>
            <mission_type>1</mission_type>
            <course>{random_target[0]}</course>
        </order>
        </operations>'''

        client_socket.sendall(f"/order/{xml}".encode())

        order_result = pd.DataFrame.from_dict(data=json.loads(recv()), orient='index')
        order_result

        client_socket.sendall("/data/unit_table".encode())

        units = pd.DataFrame.from_dict(data=json.loads(recv()), orient='index')
        units
  




    
  

In [38]:
client_socket.sendall("/result".encode())

msg = pd.DataFrame.from_dict(data=json.loads(recv()), orient='index')
msg

,0
code,200
message,Round 2 is finished.
data,"{'round': 2, 'is_win': True, 'score': 49321}"


#### Round 3

In [39]:
client_socket.sendall("/start".encode())

msg = pd.DataFrame.from_dict(data=json.loads(recv()), orient='index')
msg

,0
code,200
message,Round 3 is ready.
data,{'round': 3}


In [40]:
client_socket.sendall("/data/unit_table".encode())

units = pd.DataFrame.from_dict(data=json.loads(recv()), orient='index')
units

,0
code,200
message,Success
data,"{""D1-A"": {""Ordered"": false, ""Available"": true,..."
time,0600


In [41]:
client_socket.sendall("/data/target_list".encode())

targets = pd.DataFrame.from_dict(data=json.loads(recv()), orient='index')
targets

,0
code,200
message,Success
data,"{""Targets"": {""T1"": {""latitude"": 280, ""longitud..."
time,0600


In [42]:
Target_list = pd.DataFrame.from_dict(data=json.loads(targets[0]['data']), orient='columns')

In [43]:
Target_dict = json.loads(targets[0]['data'])
Target_dict
Target_list_temp=[]
random_target = []
for i in range(1,10):
    random_target_temp = 'T'+f'{i}'
    if(Target_dict['Targets'][random_target_temp]['probability'] != 0):
        Target_list_temp.append(Target_dict['Targets'][random_target_temp])
        random_target.append(random_target_temp)
Lake_info = Target_dict['Lakes']['L1']

In [44]:
lake_lat = Lake_info['latitude']
lake_long = Lake_info['longitude']

base_list = ['A', 'B', 'C']
base_lat ={}
base_long = {}
unit_dict = json.loads(units[0]['data'])
aircraft_info = json.loads(spec[0]['data'])
time_dict = {}
weighted_cost = {}
for i in base_list:
    base_lat[i] = Target_dict['Bases'][i]['latitude']
    base_long[i] = Target_dict['Bases'][i]['longitude']

In [45]:
xml = f'''<operations>
    <order>
        <time>0601</time>
        <base>{unit_dict[select_aircraft(Target_list_temp,3)[0][0][0]]['Base']}</base>
        <aircraft_type>{aircraft_info[select_aircraft(Target_list_temp,3)[0][0][0][0:2]]['Aircraft Type']}</aircraft_type>
        <track_number>{select_aircraft(Target_list_temp,3)[0][0][0]}</track_number>
        <mission_type>1</mission_type>
        <course>{random_target[0]}</course>
    </order>
    <order>
        <time>0602</time>
        <base>{unit_dict[select_aircraft(Target_list_temp,3)[0][1][0]]['Base']}</base>
        <aircraft_type>{aircraft_info[select_aircraft(Target_list_temp,3)[0][1][0][0:2]]['Aircraft Type']}</aircraft_type>
        <track_number>{select_aircraft(Target_list_temp,3)[0][1][0]}</track_number>
        <mission_type>1</mission_type>
        <course>{random_target[1]}</course>
    </order>
    <order>
        <time>0603</time>
        <base>{unit_dict[select_aircraft(Target_list_temp,3)[0][2][0]]['Base']}</base>
        <aircraft_type>{aircraft_info[select_aircraft(Target_list_temp,3)[0][2][0][0:2]]['Aircraft Type']}</aircraft_type>
        <track_number>{select_aircraft(Target_list_temp,3)[0][2][0]}</track_number>
        <mission_type>1</mission_type>
        <course>{random_target[2]}</course>
    </order>
    <order>
        <time>0604</time>
        <base>{unit_dict[select_aircraft(Target_list_temp,3)[0][3][0]]['Base']}</base>
        <aircraft_type>{aircraft_info[select_aircraft(Target_list_temp,3)[0][3][0][0:2]]['Aircraft Type']}</aircraft_type>
        <track_number>{select_aircraft(Target_list_temp,3)[0][3][0]}</track_number>
        <mission_type>1</mission_type>
        <course>{random_target[0]}</course>
    </order>
    <order>
        <time>0605</time>
        <base>{unit_dict[select_aircraft(Target_list_temp,3)[0][4][0]]['Base']}</base>
        <aircraft_type>{aircraft_info[select_aircraft(Target_list_temp,3)[0][4][0][0:2]]['Aircraft Type']}</aircraft_type>
        <track_number>{select_aircraft(Target_list_temp,3)[0][4][0]}</track_number>
        <mission_type>1</mission_type>
        <course>{random_target[1]}</course>
    </order>
    <order>
        <time>0606</time>
        <base>{unit_dict[select_aircraft(Target_list_temp,3)[0][5][0]]['Base']}</base>
        <aircraft_type>{aircraft_info[select_aircraft(Target_list_temp,3)[0][5][0][0:2]]['Aircraft Type']}</aircraft_type>
        <track_number>{select_aircraft(Target_list_temp,3)[0][5][0]}</track_number>
        <mission_type>1</mission_type>
        <course>{random_target[2]}</course>
    </order>
</operations>'''

In [46]:
client_socket.sendall(f"/order/{xml}".encode())

order_result = pd.DataFrame.from_dict(data=json.loads(recv()), orient='index')
order_result

,0
code,200
message,Success


In [47]:
xml_skip = '''<operations>
            </operations>'''

In [48]:
client_socket.sendall("/data/unit_table".encode())

units = pd.DataFrame.from_dict(data=json.loads(recv()), orient='index')
units

,0
code,200
message,Success
data,"{""D1-A"": {""Ordered"": false, ""Available"": true,..."
time,0601


In [49]:
aircraft_number = 6
iter_num_list = []
while True:
    check = 0
    trigger = 0
    client_socket.sendall(f"/order/{xml_skip}".encode())

    order_result = pd.DataFrame.from_dict(data=json.loads(recv()), orient='index')
    order_result

    client_socket.sendall("/data/target_list".encode())

    targets = pd.DataFrame.from_dict(data=json.loads(recv()), orient='index')
    targets
    try:
        Target_dict = json.loads(targets[0]['data'])
        Target_dict
    except:
        break
    Target_list_temp=[]
    random_target = []
    for i in range(1,10):
        random_target_temp = 'T'+f'{i}'
        if(Target_dict['Targets'][random_target_temp]['threat'] == 0):
            check = check + 1

        if(Target_dict['Targets'][random_target_temp]['probability'] != 0 and Target_dict['Targets'][random_target_temp]['targeted'] != True):
            aircraft_number = aircraft_number + 1
            if (aircraft_number >= 30):
                aircraft_number = aircraft_number % 30
            trigger = 1
            Target_list_temp.append(Target_dict['Targets'][random_target_temp])
            random_target.append(random_target_temp)
    if(check == 9):
        break
    if trigger == 1:    
        client_socket.sendall("/data/unit_table".encode())

        units = pd.DataFrame.from_dict(data=json.loads(recv()), orient='index')
        units
        
        xml = f'''<operations>
        <order>
            <time>{"0"+str(int(units[0]['time'])+1)}</time>
            <base>{unit_dict[select_aircraft(Target_list_temp,1)[0][aircraft_number][0]]['Base']}</base>
            <aircraft_type>{aircraft_info[select_aircraft(Target_list_temp,1)[0][aircraft_number][0][0:2]]['Aircraft Type']}</aircraft_type>
            <track_number>{select_aircraft(Target_list_temp,1)[0][aircraft_number][0]}</track_number>
            <mission_type>1</mission_type>
            <course>{random_target[0]}</course>
        </order>
        </operations>'''

        client_socket.sendall(f"/order/{xml}".encode())

        order_result = pd.DataFrame.from_dict(data=json.loads(recv()), orient='index')
        order_result

        client_socket.sendall("/data/unit_table".encode())

        units = pd.DataFrame.from_dict(data=json.loads(recv()), orient='index')
        units
  


In [50]:
client_socket.sendall("/result".encode())

msg = pd.DataFrame.from_dict(data=json.loads(recv()), orient='index')
msg

,0
code,200
message,Round 3 is finished.
data,"{'round': 3, 'is_win': True, 'score': 28365}"
